In [710]:
from datetime import datetime, timedelta
import pytz
from tabulate import tabulate

import requests
import xml.etree.ElementTree as ET

In [711]:
# Send departures API request with xml data payload
def send_departures_request(ns,api_key, time_now):
    # Select request params
    tree = ET.parse('API_request.xml')
    root = tree.getroot()

    RequestTimestamp = root.find('./trias:ServiceRequest/siri:RequestTimestamp',ns)
    RequestTimestamp.text = time_now.strftime("%Y-%m-%dT%H:%M:%S.000Z")

    DepArrTime = root.find('./trias:ServiceRequest/trias:RequestPayload/trias:StopEventRequest/trias:Location/trias:DepArrTime',ns)
    DepArrTime.text = time_now.strftime("%Y-%m-%dT%H:%M:%S")

    # an_element = ET.Element("Animal", Species="Python")
    xml_payload = ET.tostring(root, encoding='unicode')

    # API url
    endpoint_url = 'https://api.opentransportdata.swiss/trias2020'

    try:
        response = requests.post(endpoint_url, headers={'Content-Type': 'application/xml', 'Authorization': 'Bearer {}'.format(api_key)}, data=xml_payload)
        if response.status_code == 200:
            # Process the response data as needed
            return response
        else:
            print("Error: Unable to fetch data from the API. Status code:", response.status_code)
    except Exception as e:
        print("An error occurred:", e)

In [712]:
# Process API response and extract line_numer, direction, and departure time
def process_response(ns,response,time_now):
    XML_PATHS = {
        "responses":        "./trias:ServiceDelivery/trias:DeliveryPayload/trias:StopEventResponse",
        "line_num":         "./trias:StopEvent/trias:Service/trias:PublishedLineName/trias:Text",
        "direction":        "./trias:StopEvent/trias:Service/trias:DestinationText/trias:Text",
        "departure_time":   "./trias:StopEvent/trias:ThisCall/trias:CallAtStop/trias:ServiceDeparture/trias:TimetabledTime"
    }

    # Parse API response from string to object
    tree = ET.ElementTree(ET.fromstring(response.text))
    root = tree.getroot()

    # Get list of responses
    reponses = root.find(XML_PATHS["responses"],ns)

    # Extract critical info for each response
    departures_table = []
    for elem in reponses.findall('trias:StopEventResult',ns):
        # Get line number
        line_number = elem.find(XML_PATHS["line_num"],ns).text
        # Get direction/final station
        direction = elem.find(XML_PATHS["direction"],ns).text 
        # Get departure time
        departure_time_UTC = datetime.strptime( elem.find(XML_PATHS["departure_time"],ns).text, '%Y-%m-%dT%H:%M:%SZ')
        departure_time_UTC = departure_time_UTC.replace(tzinfo=pytz.utc)
        departure_time_ZH = departure_time_UTC.astimezone(pytz.timezone("Europe/Zurich"))
        # Convert to time left
        time_left = departure_time_ZH - time_now
        if time_left < timedelta(minutes=1):
            time_left = chr(int('1F68D',16))
        else:
            time_left = time_left

        departures_table.append(dict(line_number=line_number, direction=direction, time_left=time_left))

    header = departures_table[0].keys()
    rows =  [r.values() for r in departures_table]
    print(tabulate(rows, header))

In [713]:
# Fetch API key from txt file
api_key = xml_payload = open("api_key.txt").read()

# Define API namespaces
ns = {'trias': 'http://www.vdv.de/trias','siri': "http://www.siri.org.uk/siri" }

# Get current time
time_now = datetime.now().astimezone(pytz.timezone("Europe/Zurich"))

# Send API request for current departures
response = send_departures_request(ns, api_key, time_now)

# Process XML response and extract display info
process_response(ns, response, time_now)



  line_number  direction                       time_left
-------------  ------------------------------  --------------
           11  ZÃ¼rich, Auzelg                 0:02:32.181451
           33  ZÃ¼rich Tiefenbrunnen, Bahnhof  0:04:50.181451
           14  ZÃ¼rich, Triemli                0:05:50.181451
           14  ZÃ¼rich, Seebach                0:06:38.181451
